<a href="https://colab.research.google.com/github/sofiacompagnoni/interestUSI/blob/main/Assignment1ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/GiorgiaAuroraAdorni/ML-bachelor-course-assignments-sp23/blob/main/assignment%201/deliverable/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1
Student: Sofia Compagnoni

--- 
# IMPORTANT: all the submitted code should be in 2 cells
1) How you trained, evaluated and saved your model
2) How to load your model from a file, load the data and evaluate the model. Cell 2) should be running independently (even if cell 1 is not run)

In [ ]:
# Import libraries
import numpy as np 
from sklearn.metrics import mean_squared_error   
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error   
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression  
from scipy.stats import ttest_rel     
from keras.models import Sequential
from keras.layers import Dense  
import pickle

# Load data 
data = np.load("data.npz")

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = data['x']

# y is a Numpy array of shape (n_samples, ) with the targets
y = data['y']

# Check
print(x[:5])

# Split data into train and test

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, shuffle=True, random_state=1)
print('train_data shape:', x_train.shape, y_train.shape)
print('test_data shape:', x_test.shape, y_test.shape)

# T1

X = np.column_stack((np.ones_like(x_train[:, 0]), x_train[:, 0], x_train[:, 1], np.sin(x_train[:, 1]), x_train[:, 0] * x_train[:, 1]))

theta_hat = np.linalg.lstsq(X, y_train, rcond=None)[0]
print('Parameters:',theta_hat)

# Define the model (linearregression) f(x, theta)
def linearregression(x, theta):
    y = theta[0] + theta[1] * x[:, 0] + theta[2] * x[:, 1] + theta[3] * np.sin(x[:, 1]) + theta[4] * x[:, 0] * x[:, 1]
    return y

test_pred_lr = linearregression(x_test, theta_hat)
mse = mean_squared_error(y_test, test_pred_lr)
print('MSE test Linear regression:', mse)

train_pred_lr = linearregression(x_train, theta_hat)
mse1 = mean_squared_error(y_train, train_pred_lr)
print('MSE train Linear regression:', mse1)

with open('modelT1.pkl','wb') as f:
  pickle.dump(linearregression,f)

# T2

poly = PolynomialFeatures(degree=3)
x_train_poly = poly.fit_transform(x_train)
x_test_poly = poly.transform(x_test)

model = LinearRegression()
model.fit(x_train_poly, y_train)

pred_test_poly = model.predict(x_test_poly)
mse2 = mean_squared_error(y_test, pred_test_poly)

pred_train_poly = model.predict(x_train_poly)
mse3 = mean_squared_error(y_train, pred_train_poly)

print('MSE test poly:', mse2)
print('MSE train poly:', mse3)

# T-test

t_statistic, p_value = ttest_rel(pred_test_poly, test_pred_lr)
if p_value < 0.05:
    print("The difference between the MSEs is statistically significant")
else:
    print("The difference between the MSEs is not statistically significant")

with open('modelT2.pkl','wb') as f:
  pickle.dump(poly,f)

# T3 (Bonus)

model = Sequential()
model.add(Dense(64, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')

model.fit(x_train, y_train, validation_split=0.2, epochs=280, batch_size=32, verbose=0)

y_pred_nn = model.predict(x_test)
mse4 = mean_squared_error(y_test, y_pred_nn)

print('MSE: ', mse4)

with open('modelT3.pkl','wb') as f:
  pickle.dump(model,f)

[[3.91981894 0.28059897]
 [2.63634368 1.62935366]
 [0.58107833 4.64420079]
 [4.59640668 2.3688777 ]
 [4.06567427 3.55362318]]
train_data shape: (1400, 2) (1400,)
test_data shape: (600, 2) (600,)
Parameters: [ 1.3236108  -0.05958542 -0.57665406  0.4393116   0.04132068]
MSE test Linear regression: 0.6804349242489404
MSE train Linear regression: 0.7264846428520856
MSE test poly: 0.4633805024103428
MSE train poly: 0.4943805577461204
The difference between the MSEs is not statistically significant
19/19 [==============================] - 0s 970us/step
MSE:  0.014742092058230778


# Example on how to use baseline model:

In [ ]:
# Import libraries
import pickle
import io
import requests
import numpy as np

def evaluate_predictions(y_true, y_pred):
    """
    Evaluates the mean squared error between the values in y_true and the values
    in y_pred.
    ### YOU CAN NOT EDIT THIS FUNCTION ###
    :param y_true: Numpy array, the true target values from the test set;
    :param y_pred: Numpy array, the values predicted by your model.
    :return: float, the mean squared error between the two arrays.
    """
    assert y_true.shape == y_pred.shape
    return ((y_true - y_pred) ** 2).mean()


def load_model(filename):

    with open(filename, 'rb') as f:
        model = pickle.load(f)

    return model

# Load the data
# Load data 
data = np.load("data.npz")

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = data['x']

# y is a Numpy array of shape (n_samples, ) with the targets
y = data['y']

# Load the trained model
model_path = './modelT3.pkl'
model = load_model(model_path)

y = y.reshape(-1,1)


y_pred = model.predict(x)

############################################################################
# STOP EDITABLE SECTION: do not modify anything below this point.
############################################################################

# Evaluate the prediction using MSE
mse = evaluate_predictions(y_pred, y)
print(f'MSE on whole dataset: {mse}')

# NOTE: NOW THIS CELL IS NOT WORKING SINCE YOU NEED TO CHANGE THE INPUT.
# DO IT AND EVERYTHING RUNS SMOOTH


63/63 [==============================] - 0s 2ms/step
MSE on whole dataset: 0.014962979877481967
